Semtimental Analysis of Fake News Classifier using LSTM

In [25]:
import pandas as pd
df = pd.read_csv('fake-news/train.csv')

In [26]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [31]:
df.shape

(20800, 5)

In [30]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [34]:
df = df.dropna()

In [35]:
X= df.drop('label', axis = 1)

In [36]:
y = df['label']

In [37]:
X.shape

(18285, 4)

In [38]:
y.shape

(18285,)

In [39]:
import tensorflow as tf

In [40]:
tf.__version__

'2.17.0'

In [44]:
from tensorflow.keras.layers import Embedding  # for word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [45]:
voc_size = 5000

In [46]:
msg = X.copy()

In [47]:
msg['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [49]:
# msg.reset_index(inplace = True)

In [50]:
msg

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [51]:
import nltk
import re #regular expression
from nltk.corpus import stopwords

In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/kuki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data PreProcessing

In [56]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]
for i in range (0,len(msg)):
    review = re.sub('[^a-zA-Z]', ' ', msg['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [57]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [58]:
onehot_repr=[one_hot(words, voc_size) for words in corpus]
onehot_repr

[[112, 1403, 185, 4855, 3050, 4532, 2821, 3608, 1440, 3996],
 [2723, 1877, 2399, 3294, 3445, 4262, 147],
 [3969, 4271, 3106, 4569],
 [898, 402, 4159, 2395, 3973, 1865],
 [2564, 3445, 4432, 4654, 4525, 3905, 3445, 2531, 3238, 4669],
 [2433,
  4361,
  1276,
  1953,
  1751,
  2397,
  516,
  4601,
  2707,
  4723,
  982,
  1236,
  4648,
  1063,
  147],
 [2022, 840, 403, 167, 3561, 3956, 570, 4392, 499, 3025, 1967],
 [318, 766, 1372, 3165, 168, 3025, 2397, 1241, 499, 3025, 1967],
 [117, 3401, 1996, 4557, 4234, 123, 2012, 1925, 2397, 3441],
 [4027, 1065, 1874, 422, 2605, 3391, 771, 4398],
 [1143, 3494, 4691, 268, 405, 1012, 3062, 1813, 2927, 963, 2721],
 [2395, 610, 3050, 123, 2397, 168],
 [3184, 3580, 3733, 2548, 4424, 2711, 2265, 81, 3289],
 [2247, 1418, 3491, 2012, 308, 644, 910, 499, 3025, 1967],
 [4175, 4620, 70, 285, 3979, 499, 3025, 1967],
 [2452, 3573, 1767, 3243, 1780, 3828, 4620, 2368, 2400, 432],
 [1068, 4055, 1877],
 [3720, 2375, 4636, 1528, 2397, 4569, 2662, 147],
 [3431, 4314, 2

In [62]:
onehot_repr[1]

[2723, 1877, 2399, 3294, 3445, 4262, 147]

Embedding Representation

In [68]:

msg_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen = msg_length)


In [69]:
embedded_docs

array([[   0,    0,    0, ..., 3608, 1440, 3996],
       [   0,    0,    0, ..., 3445, 4262,  147],
       [   0,    0,    0, ..., 4271, 3106, 4569],
       ...,
       [   0,    0,    0, ...,  499, 3025, 1967],
       [   0,    0,    0, ..., 3355,  223, 3511],
       [   0,    0,    0, ..., 4634, 3442, 1132]], dtype=int32)

In [72]:
#creating a model
embedding_vector_features = 40   
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = msg_length))
model.add(LSTM(100)) #lstm with 100 neurons
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [112]:
print(model.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 40)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,505 (2.94 MB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 513,004 (1.96 MB)

None


In [75]:
import numpy as np
X_final = np.array(embedded_docs)  #converting into array for training purpose
y_final=np.array(y)

In [76]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X_final, y_final, test_size= 0.33, random_state  =42)

Model Training

In [80]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=10, batch_size = 64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 16s 60ms/step - accuracy: 0.8147 - loss: 0.4178 - val_accuracy: 0.9140 - val_loss: 0.1965
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 62ms/step - accuracy: 0.9371 - loss: 0.1512 - val_accuracy: 0.9201 - val_loss: 0.1890
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 19s 56ms/step - accuracy: 0.9633 - loss: 0.1039 - val_accuracy: 0.9211 - val_loss: 0.2062
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.9781 - loss: 0.0655 - val_accuracy: 0.9216 - val_loss: 0.2213
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 60ms/step - accuracy: 0.9853 - loss: 0.0434 - val_accuracy: 0.9211 - val_loss: 0.2737
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 62ms/step - accuracy: 0.9923 - loss: 0.0256 - val_accuracy: 0.9158 - val_loss: 0.3294
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 61ms/step - accuracy: 0.9944 - loss: 0.0176 - val_accuracy: 0.9208 - val_loss: 0.4197
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.9981 - loss: 0.0075 - 

In [82]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step


In [83]:
y_pred = np.where(y_pred>0.6, 1, 0) #you can check the threshold by AUC, ROC Curve

In [85]:
from sklearn.metrics import confusion_matrix

In [87]:
confusion_matrix(y_test, y_pred)

array([[3130,  289],
       [ 237, 2379]])

In [88]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9128417564208782

In [90]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



Adding Dropout

In [81]:
from tensorflow.keras.layers import Dropout

In [91]:
#model creation
embedding_vector_features = 40   
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = msg_length))
model.add(Dropout(0,3))
model.add(LSTM(100))
model.add(Dropout(0,3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

/home/kuki/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [92]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=10, batch_size = 64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 15s 56ms/step - accuracy: 0.8089 - loss: 0.4152 - val_accuracy: 0.9183 - val_loss: 0.1908
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.9421 - loss: 0.1449 - val_accuracy: 0.9210 - val_loss: 0.1939
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.9625 - loss: 0.1030 - val_accuracy: 0.9211 - val_loss: 0.2359
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step - accuracy: 0.9740 - loss: 0.0747 - val_accuracy: 0.9201 - val_loss: 0.2328
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.9805 - loss: 0.0562 - val_accuracy: 0.9181 - val_loss: 0.2947
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step - accuracy: 0.9833 - loss: 0.0448 - val_accuracy: 0.9110 - val_loss: 0.2919
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 61ms/step - accuracy: 0.9903 - loss: 0.0288 - val_accuracy: 0.9138 - val_loss: 0.4423
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - accuracy: 0.9948 - loss: 0.0197 - 

In [93]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step


In [94]:
y_pred = np.where(y_pred > 0.6, 1, 0)

In [106]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print(confusion_matrix(y_pred, y_test))


[[3141  242]
 [ 278 2374]]


In [108]:
acc = accuracy_score(y_pred, y_test)
acc

0.9138359569179785

In [111]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      3383
           1       0.91      0.90      0.90      2652

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

